In [1]:
import numpy as np
import pandas as pd
from boruta import BorutaPy
from sklearn.metrics import accuracy_score 
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [2]:
%cd Z:\Project_Top2B\April_2023\regionwise\frontal_lobe\Healthy
data = pd.read_csv('Merged_Top2B_Frontal.csv')
data = data.drop(['Sample_ID'], axis = 1)
data.head(1)

Z:\Project_Top2B\April_2023\regionwise\frontal_lobe\Healthy


,Age,cg00005072,cg00010266,cg00017203,cg00017489,cg00018024,cg00018539,cg00019511,cg00021539,cg00029353,...,cg27597926,cg27598168,cg27601855,cg27624319,cg27629898,cg27633533,cg27635069,cg27658416,cg27658967,cg27664120
0,0.002739,0.818356,0.079312,0.851736,0.159355,0.058467,0.869852,0.836981,0.102515,0.060177,...,0.088717,0.912107,0.2732,0.273216,0.856758,0.927893,0.457576,0.213621,0.073548,0.873596


In [3]:
# Replace '' values with NaN
data = data.replace('', np.nan)

# Loop over each column and replace NaN values with column means
for col in data.columns:
    # Convert the column to numeric data type
    data[col] = pd.to_numeric(data[col])
    
    # Calculate the mean of the column
    col_mean = data[col].mean()
    
    # Replace NaN values with the column mean
    data[col].fillna(col_mean, inplace=True)

data.to_csv("Merged_Top2B_Mean_fl.csv", index = False)
data.head(2)

,Age,cg00005072,cg00010266,cg00017203,cg00017489,cg00018024,cg00018539,cg00019511,cg00021539,cg00029353,...,cg27597926,cg27598168,cg27601855,cg27624319,cg27629898,cg27633533,cg27635069,cg27658416,cg27658967,cg27664120
0,0.002739,0.818356,0.079312,0.851736,0.159355,0.058467,0.869852,0.836981,0.102515,0.060177,...,0.088717,0.912107,0.273200,0.273216,0.856758,0.927893,0.457576,0.213621,0.073548,0.873596
1,0.005479,0.768641,0.078670,0.801902,0.136464,0.138156,0.865317,0.852079,0.105216,0.068154,...,0.112448,0.876858,0.160398,0.473848,0.798062,0.929042,0.347084,0.097016,0.088556,0.879235


In [4]:
X = data.drop(['Age'], axis = 1)
y = data['Age']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 1) 

In [5]:
rf_all_features = RandomForestRegressor(random_state=1)
rf_all_features.fit(X_train, y_train)

RandomForestRegressor(random_state=1)

accuracy_score(y_test, rf_all_features.predict(X_test))

In [6]:
rfc = RandomForestRegressor(random_state=1)
boruta_selector = BorutaPy(rfc, n_estimators= 'auto', verbose=1, random_state=1)
boruta_selector.fit(np.array(X_train), np.array(y_train))  

Iteration: 1 / 100
Iteration: 2 / 100
Iteration: 3 / 100
Iteration: 4 / 100
Iteration: 5 / 100
Iteration: 6 / 100
Iteration: 7 / 100
Iteration: 8 / 100
Iteration: 9 / 100
Iteration: 10 / 100
Iteration: 11 / 100
Iteration: 12 / 100
Iteration: 13 / 100
Iteration: 14 / 100
Iteration: 15 / 100
Iteration: 16 / 100
Iteration: 17 / 100
Iteration: 18 / 100
Iteration: 19 / 100
Iteration: 20 / 100
Iteration: 21 / 100
Iteration: 22 / 100
Iteration: 23 / 100
Iteration: 24 / 100
Iteration: 25 / 100
Iteration: 26 / 100
Iteration: 27 / 100
Iteration: 28 / 100
Iteration: 29 / 100
Iteration: 30 / 100
Iteration: 31 / 100
Iteration: 32 / 100
Iteration: 33 / 100
Iteration: 34 / 100
Iteration: 35 / 100
Iteration: 36 / 100
Iteration: 37 / 100
Iteration: 38 / 100
Iteration: 39 / 100
Iteration: 40 / 100
Iteration: 41 / 100
Iteration: 42 / 100
Iteration: 43 / 100
Iteration: 44 / 100
Iteration: 45 / 100
Iteration: 46 / 100
Iteration: 47 / 100
Iteration: 48 / 100
Iteration: 49 / 100
Iteration: 50 / 100
Iteration

BorutaPy(estimator=RandomForestRegressor(n_estimators=225,
                                         random_state=RandomState(MT19937) at 0x1CCAB98E640),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x1CCAB98E640, verbose=1)

In [7]:
# print("Ranking: ",boruta_selector.ranking_)          
print("No. of significant features: ", boruta_selector.n_features_)

No. of significant features:  147


In [8]:
selected_rf_features = pd.DataFrame({'CpG':list(X_train.columns),
                                       'Ranking':boruta_selector.ranking_})
selected_rf_features = selected_rf_features.sort_values(by='Ranking')

In [9]:
selected_rf_features.to_csv ('CpG_BorutaPy_onAll_fl.csv', index = False)